![Bellabeat](http://play-lh.googleusercontent.com/1DEgw7f-f8Dtp7r0lZ3qn7FfsNb_zYGWVkrAdf5ht8eDFEnRi1HX5Qk-NRTJ9cwbzUg)

# Introduction - Bellabeat

**How Can a Wellness Technology Company Play it Smart?**

Bellabeat empowers women worldwide by leveraging innovative technology to track and enhance overall health, facilitating a strong mind-body connection throughout various life stages. As a tech-driven company specializing in health-focused smart products, Bellabeat offers a variety of devices that monitor activity, sleep, stress, and reproductive health, providing women with valuable insights into their well-being.
The primary goal of this case study is to analyze fitness data from smart devices to identify potential growth opportunities for Bellabeat, with a particular focus on one of their key products: the Bellabeat app.

Key Stakeholders:

•	Urška Sršen: Bellabeat’s cofounder and Chief Creative Officer

•	Sando Mur: Bellabeat cofounder; key member of Bellabeat executive team

•	Bellabeat Marketing Analytics team


# Step 1: Ask

1. What are some trends in smart device usage? 
2. How could these trends apply to Bellabeat customers? 
3. How could these trends help influence Bellabeat marketing strategy?





# Step 2: Prepare 

The data source for this case study is the FitBit Fitness Tracker Data, a comprehensive dataset that provides valuable insights into various fitness and activity metrics. This dataset is hosted on Kaggle and made available under the CC0 Public Domain license by [Mobius](http://https://www.kaggle.com/arashnic). It consists of 18 .csv files, containing detailed information on activity metrics, calories burned, sleep records, metabolic equivalent of tasks (METs), heart rate, and steps. These metrics are recorded across different timeframes, including seconds, minutes, hours, and days, allowing for a granular analysis of fitness behaviors.

By leveraging this dataset, the case study aims to explore patterns and trends in fitness activities, offering a deeper understanding of how individuals engage with their routines and the impact on their overall health.
However, it is important to note some limitations that could affect the results of the analysis:

-Missing Demographics: The dataset lacks demographic information, which may limit the ability to analyze trends based on age, gender, or other factors.

Small Sample Size: The limited number of participants in the dataset may not fully represent the broader population, potentially impacting the generalizability of the findings.

-Short Data Collection Period: The data was collected over a relatively short period, which may not capture long-term trends or seasonal variations.

*These limitations should be taken into account when interpreting the results of the analysis.*


# Step 3: Process


**Loading Libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file 
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns

from pandas.api.types import CategoricalDtype


***Importing Datasets***

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


***Importing the dataset "daily_activity_merged.csv" using the pandas pd.read.csv() function***

In [ ]:
df = pd.read_csv('/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv')

***Data Exploration***

In [ ]:
df.shape

We have 940 rows and 15 columns in the dataset. Let's examine the column names using the `.columns` function from pandas. 

In [ ]:
df.columns

With the column names in hand, let's take a quick look at the rows and data using the `.head()` function from pandas.

In [ ]:
df.head(10)

The dataset tracks daily data collected by FitBit fitness tracking devices, such as smartwatches and fitness apps. Here is a summary of the columns:

- **Id**: A unique identifier for each user in the survey.
- **ActivityDate**: The specific date of the entry.
- **TotalSteps**: The total number of steps taken by each user each day.
- **TotalDistance**: The total distance traveled by each user each day.
- **TrackerDistance**: The distance recorded by the device each day.
- **LoggedActivitiesDistance**: The distance tracked by the device for specific activities.
- **VeryActiveDistance**: The distance traveled during periods of intense physical activity.
- **ModeratelyActiveDistance**: The distance traveled during periods of moderate physical activity.
- **LightActiveDistance**: The distance traveled during periods of light physical activity.
- **SedentaryActiveDistance**: The distance traveled while in a sedentary state.
- **VeryActiveMinutes**: The minutes spent in intense physical activity.
- **FairlyActiveMinutes**: The minutes spent in moderate physical activity.
- **LightlyActiveMinutes**: The minutes spent in light physical activity.
- **SedentaryMinutes**: The minutes spent in a sedentary state.
- **Calories**: The number of calories burned on that specific day.

After reviewing the columns and data, we can begin the data cleaning process.

***Clean the data**

First, we should verify that the data types match the content and purpose of each column. We can use the `.dtypes` function to check this.

In [ ]:
df.dtypes

We observe that the **Id** column is currently an integer, but it should be a string or object because it serves only as an identifier and is not intended for mathematical operations such as sums or multiplications.

Additionally, the **ActivityDate** column is currently an object but should be formatted as a Date.

Apart from these two columns, the data types of the remaining columns appear to be correct.

In [ ]:
df['Id'] = df['Id'].astype(str)
df['ActivityDate'] = pd.to_datetime(df['ActivityDate'], format='%m/%d/%Y')
df.dtypes # a recheck will be done after reorganization.

We will now address the formatting issues by converting the **Id** column from an integer to a string using the `.astype(str)` function, and changing the **ActivityDate** column from an object (or string) to a datetime format.

***Checking Column Values***

Next, we will examine the relevance of each column for our analysis. Initially, we note that the **TotalDistance** column and other distance-related columns may be redundant. At first glance, **TotalDistance** and **TrackerDistance** appear to have similar values, but we need to confirm this. Additionally, we will investigate whether **TrackerDistance** or **TotalDistance** is the sum of the various "*ActiveDistance" columns, as this assumption may need verification.

In [ ]:
# We will create a new column by summing the "ActiveDistance" columns to verify if it matches the **TotalDistance** or **TrackerDistance** columns.

df['sum_distance'] = df['VeryActiveDistance'] + df['ModeratelyActiveDistance'] + df['LightActiveDistance'] + df['SedentaryActiveDistance']

# We also observe that **LoggedActivitiesDistance** predominantly contains values of 0.0. We will filter the data to identify the entries where the value is greater than 0.

df.loc[(df['LoggedActivitiesDistance'] > 0),['TotalDistance','TrackerDistance','LoggedActivitiesDistance','sum_distance']]

The previous analysis indicates that while **TotalDistance** and **TrackerDistance** are not always identical, they are very similar in most cases. We also observed that **LoggedActivitiesDistance** has some non-zero entries, though these are infrequent. Additionally, the sum of the **ActiveDistance** columns closely matches the **TotalDistance**, with only minor differences due to rounding.

Given this, we need to decide whether to keep or remove some columns. Since **TotalDistance** and **TrackerDistance** are almost always similar, with **TotalDistance** generally having higher values, we will retain the **TotalDistance** column.

Regarding the **ActiveDistance** columns, the exact criteria for categorization (e.g., the difference between 'Moderately Active' and 'Very Active') are unclear without further information, such as heart rate or steps per minute. Despite this uncertainty, we will keep these columns.

Similarly, the **ActiveMinutes** columns will be retained and combined into a new column for analysis.

In [ ]:
df['TotalMinutes'] = df['VeryActiveMinutes'] + df['FairlyActiveMinutes'] + df['LightlyActiveMinutes'] + df['SedentaryMinutes']

Next, we'll rename the columns using the `rename` function and convert all column names to lowercase using the `str.lower()` function.

In [ ]:
df.columns = df.columns.str.lower()
df.rename(columns = {'trackerdistance':'tracker_distance','activitydate':'activity_date','totalsteps':'total_steps','totaldistance':'total_distance',
       'loggedactivitiesdistance':'logged_activities_distance', 'veryactivedistance':'very_active_distance',
       'moderatelyactivedistance':'moderately_active_distance', 'lightactivedistance':'light_active_distance',
       'sedentaryactivedistance':'sedentary_active_distance', 'veryactiveminutes':'very_active_minutes',
       'fairlyactiveminutes':'fairly_active_minutes','lightlyactiveminutes':'lightly_active_minutes',
       'sedentaryminutes':'sedentary_minutes'}
         ,inplace=True) 
print('check the name of the columns:')
df.columns

***Creating Columns***

Let's add a column to indicate the day of the week using the `day_name()` function, and another column to show the day number of the week using the `weekday()` function.

In [ ]:
day_of_week = df['activity_date'].dt.day_name()
df['day_of_week'] = day_of_week

df['n_day_of_week'] = df['activity_date'].dt.weekday # 0 monday 6 sunday

***Check for empty cell and null values***

Check for empty cells and null values using the `isna()` function in combination with `sum()`.

In [ ]:
print('Total number of null values are: ')
print(df.isna().sum())

***Checking for duplicate entries using the `duplicated().sum()` function.*****

In [ ]:
print('Total number of duplicated values are: ',df.duplicated().sum())

As noted above, there are no duplicated or null values.

***Subset the data***

Now, we can select only the columns that will be used for our analysis. 

In [ ]:
df = df[['id', 'activity_date', 'total_steps', 'total_distance',
       #'tracker_distance', 'logged_activities_distance',
       #'very_active_distance', 'moderately_active_distance',
       #'light_active_distance', 'sedentary_active_distance',
       'very_active_minutes', 'fairly_active_minutes',
       'lightly_active_minutes', 'sedentary_minutes', 'calories',
       #'sum_distance','totalminutes', 
       'day_of_week', 'n_day_of_week'
        ]].copy()

**Category Creation**

Next, I will create my own user categorization based on physical activity levels and device usage:

**Physical Activity Levels:**
- **Sedentary:** Fewer than 5,000 daily steps on average
- **Active:** Between 5,000 and 10,000 daily steps on average
- **Very Active:** More than 10,000 daily steps on average

**Device Usage:**
- **Low Use:** Less than 8 hours of use per day
- **Normal Use:** Between 8 and 16 hours of use per day
- **High Use:** More than 16 hours of use per day

For this dataset, I will initially create the **activity_level** category. I will address device usage categorization when analyzing a dataset with hourly data.

In [ ]:
# Group the data by id:
id_grp = df.groupby(['id'])

# Search for average amount of steps and sort the results in descending order:
id_avg_step = id_grp['total_steps'].mean().sort_values(ascending=False)

# Afterward, I will convert the results into a DataFrame:
id_avg_step = id_avg_step.to_frame()


#I want to create a new column that categorizes each user based on their average of daily steps. The conditions for categorization are:
conditions = [
    (id_avg_step <=5000),
    (id_avg_step > 5000) & (id_avg_step < 10000),
    (id_avg_step >= 10000)
] # These are the conditions

values = ['sedentary','active','very_active'] # name of the values:

#I create a column using the `np.select` function from NumPy to assign each ID to a category:
id_avg_step['activity_level'] = np.select(conditions,values)

#I store the results in a variable for use in the next step:
id_activity_level = id_avg_step['activity_level']

#I use a list comprehension to create the **activity_level** column in the original dataset.This list comprehension retrieves the categories where the index matches the **id** column:
df['activity_level'] = [id_activity_level[c] for c in df['id']]

# Step 4: Analyze


Let's determine the number of unique IDs using the `nunique()` function, and identify the unique IDs using the `unique()` function.

In [ ]:
print('Number of unique values in id column:',df['id'].nunique())
print()
print('List of id values:',df['id'].unique())

Let's check how many times each ID appears in the dataset using the `value_counts()` function.

In [ ]:
print('How many times each id appears in the dataset?')
print(df['id'].value_counts())

As observed, there are 33 unique IDs (or users), with most appearing 31 times throughout the dataset, and some appearing fewer times.

Next, let's examine the date column to determine the minimum and maximum dates, the number of days between them, and the number of unique dates.

In [ ]:
print('The min date is:',min(df['activity_date']))
print('The max date is:',max(df['activity_date']))
print('The number of unique dates are:',df['activity_date'].nunique())

We can see that the dataset covers exactly 31 days, ranging from '2016-04-12' to '2016-05-12'.

Now we can begin the exploratory data analysis.

In [ ]:
#First, we'll use the `describe()` function to view some summary statistics:

df.describe()

Here, we can observe the mean (average), minimum and maximum values, the 50th percentile (median), and other statistics.

Notably, the maximum values indicate that someone walked 28 miles and burned 4,900 calories. These figures could be outliers, so we may need to investigate them further later on.

# Step 5: Share 


***What is the correlation between the number of steps taken and the number of calories burned?*****

In [ ]:
ax =sns.scatterplot(x='total_steps', y='calories', data=df,hue='activity_level')

#handles, labels = ax.get_legend_handles_labels()
#plt.legend(handles, day_of_week, fontsize=7)
plt.title('Correlation Calories vs. Steps')

plt.show()

The scatterplot reveals a positive correlation: as the number of steps increases, the number of calories burned also tends to increase. We have differentiated the data points by color according to the **activity_level** category, allowing us to see the representation of each activity level group.

**Average number steps per day**

In [ ]:
day_of_week = ['Monday','Tuesday','Wednesday','Thursday', 'Friday','Saturday','Sunday']
fig, ax =plt.subplots(1,1,figsize=(9,6))

day_grp = df.groupby(['day_of_week'])
avg_daily_steps= day_grp['total_steps'].mean()
avg_steps = df['total_steps'].mean()

plt.bar(avg_daily_steps.index,avg_daily_steps)

ax.set_xticks(range(len(day_of_week)))
ax.set_xticklabels(day_of_week)

ax.axhline(y=avg_daily_steps.mean(),color='red', label='Average daily steps')
ax.set_ylabel('Number of steps')
ax.set_xlabel('Day of the week')
ax.set_title('Avg Number of steps per day')

plt.legend()
plt.show()

The results indicate that users were most physically active on Monday, Tuesday, and Saturday, with these days showing an above-average number of steps overall. Conversely, Wednesday, Thursday, and Friday had below-average activity levels, although they were relatively similar to each other. Sunday was the least active day of the week.

From this information, we can infer that users tend to be more physically active at the beginning of the week and on Saturdays, which may suggest specific activities or routines on these days.

**Percentage in activity in minutes**

In [ ]:
very_active_mins = df['very_active_minutes'].sum() 
fairly_active_mins = df['fairly_active_minutes'].sum()
lightly_active_mins = df['lightly_active_minutes'].sum()
sedentary_mins = df['sedentary_minutes'].sum()

slices = [very_active_mins,fairly_active_mins,lightly_active_mins,sedentary_mins]
labels = ['very active minutes','fairly active minutes','lightly active minutes','sedentary minutes']
explode = [0,0,0,0.1]
plt.pie(slices, labels = labels, explode = explode, autopct='%1.1f%%',textprops=dict(size=12), shadow=True)

plt.title('Percentage of activity in minutes',fontsize=20)
plt.tight_layout()

plt.show()

This pie chart illustrates that users are mostly in a sedentary state, with a small fraction of time spent in light activity and only 2% of the time engaged in vigorous exercise.

**Correlation between activity level minutes and calories**

In [ ]:
n_day_of_week = [0,1,2,3,4,5,6]

fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(11,15),dpi=70)

sns.scatterplot(data=df,x='calories',y='sedentary_minutes',hue='activity_level',ax=axes[0,0],legend=False)

sns.scatterplot(data=df,x='calories',y='lightly_active_minutes',hue='activity_level',ax=axes[0,1],legend=False)

sns.scatterplot(data=df,x='calories',y='fairly_active_minutes',hue='activity_level',ax=axes[1,0],legend=False)

sns.scatterplot(data=df,x='calories',y='very_active_minutes',hue='activity_level',ax=axes[1,1])


plt.legend(title='Activity level',title_fontsize=18,bbox_to_anchor=(1.8,2.2),fontsize=18,frameon=True,scatterpoints=1)
fig.suptitle('Correlation between Activity level minutes and Calories',x=0.5,y=0.92,fontsize=26)
plt.show()

# Step 6:Act

After analyzing the FitBit Fitness Tracker Data, we have uncovered insights that could inform and enhance Bellabeat's marketing strategy.

**Target Audience**

Our target audience consists of women who work full-time jobs and spend significant time at the computer, in meetings, or focused on their work, as indicated by the sedentary time data. 

These women engage in some light activity to maintain their health, based on the activity type analysis. However, they need to increase their daily activity levels to gain greater health benefits. They may benefit from guidance on developing healthy habits or motivation to stay active.

Since the dataset lacks gender information, it is assumed that all genders are represented and balanced.

**Key Message for the Bellabeat Online Campaign**

The Bellabeat app is more than just a fitness tracker—it's a supportive guide that helps women balance their personal and professional lives with healthy habits. By providing daily recommendations and motivation, the app empowers users to integrate wellness into their routines effectively.

**Ideas for the Bellabeat App**

1. **Step Goals and Health Benefits:** The marketing team should emphasize the importance of regular exercise for maintaining overall health and well-being. The average daily step count is 7,638, slightly below the CDC’s recommended 10,000 steps for optimal health benefits.Bellabeat could encourage users to reach at least 10,000 steps per day by highlighting the associated health benefits, such as a 53% lower risk of all-cause mortality. [Preventing Chronic Disease, Volume 14, E83 issue](http://www.cdc.gov/pcd/issues/2016/pdf/16_0111.pdf)

2. **Calorie Management for Weight Loss:** For users looking to lose weight, Bellabeat could offer personalized suggestions for healthy, low-calorie meals for lunch and dinner, aiding in effective daily calorie management.

3. **Sleep Improvement Notifications:** To help users improve their sleep, Bellabeat could utilize app notifications to remind users when it's time to go to bed, promoting better sleep habits.

4. **Activity Reminders:** Given that most user activity occurs between 5 PM and 7 PM, possibly due to gym sessions or walks after work, Bellabeat could send reminders and motivational prompts during this time to encourage users to go for a run or walk.

5. **Reducing Sedentary Time:** As a sleep improvement strategy, Bellabeat could recommend reducing sedentary time, helping users stay active throughout the day to enhance overall sleep quality.